In [29]:
from __future__ import print_function
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [30]:
from __future__ import print_function
import pandas as pd
import os

class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        raw_notes = pd.read_csv('data/{0}_notes_generated.csv'.format(key), converters={'0': lambda x: str(x)}).values
        notes = [row[0] for row in raw_notes]
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.notes = notes
        self.extension = music_file.split('.')[1]

def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.notes)):
        row = song.notes[i]
        print (row, file=output_stepfile)
        if i % 48 == 48 - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def write_song_steps(song):
    if song.name + '.sm' in os.listdir(song.folder) and not song.name + '.sm.backup' in os.listdir(song.folder):
        os.rename(song.stepfile_name, song.stepfile_name + '.backup')
            
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()

def write_song_steps_by_key(key):
    pack, song = key.split('~')
    folder = 'StepMania/Songs/{0}/{1}/'.format(pack, song)
    stepfile = folder + '/{0}.sm'.format(song)
    music = folder + [file for file in listdir(folder) if file.endswith('.ogg') or file.endswith('.mp3')][0]

    write_song_steps(SongFile(key, folder, stepfile, music))

In [ ]:
# make two separate classifiers, one for note importance, then another to take importance and previous notes/importance and 
# output the notes

# can either go back and include prev samples to get importance or atual notes

In [67]:
samples_back_included = 8
num_classes = 5
num_features = 40 + 2
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return np.concatenate((beat_features[index], get_steps_for_index(notes, index - 1)))

def get_steps_for_index(notes, index):
    if index < 0:
        return (0, 0)
    row = notes[index][0]
    return [row.count('1'), row.count('M')]

def get_class_for_index(notes, index):
    steps, mines = get_steps_for_index(notes, index)
    if mines > 0:
        return 4
    return min(steps, 3)
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 20) != 0) and not (row_y == 1 and random.randint(0, 3) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

def build_batch_generator():
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use:
        yield (get_features_for_song(song_data[0]))

# Total 243 songs
def build_training_data(songs_start, songs_end, is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[songs_start:songs_end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return X, y

In [68]:
outputs = ['0000', '1000', '1100', '1111', 'MMMM']
def prediction_to_output_class(row):
    return outputs[row]

def prediction_to_output_one_hot(row):
    return outputs[np.argmax(row)]

def step_song(key, clf, prediction_to_output):
    song_X, song_y = get_full_features_for_song(key)
    new_song_y = clf.predict(song_X)
    new_song_output = [prediction_to_output(row) for row in new_song_y]
    
    #print ('Length: ' + str(len(new_song_y)))
    #plt.plot([new_song_y[i] for i in range(len(new_song_y)) if i % 12 == 0])
    #plt.show()
    
    pd.DataFrame(new_song_output).to_csv('data/{0}_notes_generated.csv'.format(key), index=False)
    write_song_steps_by_key(key)

In [69]:
# start with each input secion maps to one note
# train model for that (just list comprhension on noets for contains 1 maps to true)

# then move to bars eg section of 4 bars maps to output for each note
# error = probability of note being true vs was it really

# try bar + prev notes (home use weird dimensioned data?) to predict next notes

# try feeding in non structured data (bpm, position of time in song, song length, 
# things about feel of song (generated features))
# try using keras merge layer to add extra features

In [70]:
X_train, y_train = build_training_data(0, 200)
X_test, y_test = build_training_data(200, 243)

In [54]:
X_train_full, y_train_full = build_training_data(0, 200, True)
X_test_full, y_test_full = build_training_data(200, 243, True)

In [55]:
print (len(X_train))
print (len(X_train_full))

30384
540031


In [21]:
clf_rf = RandomForestClassifier(n_estimators=20)
clf_rf.fit(X, y)

#for song in ['Anubis', 'Bend Your Mind', 'Boogie Down', 'Bouff', 'Bubble Dancer']:
#    step_song('In The Groove~{0}'.format(song), clf_rf, prediction_to_output_class)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
beat_feature_model = Sequential()

beat_feature_model.add(Dense(500, input_dim=num_features_total, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(num_classes, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('softmax'))


beat_feature_model.compile(loss='categorical_crossentropy',
                           optimizer='adadelta',
                           metrics=['accuracy'])

In [72]:
# Try building one network to output importance of beats, another to output song from that and prev notes
# Also start building sequentially not off of stepfile and test on new songs

In [73]:
y_one_hot = np.zeros((len(y_train), num_classes))
y_one_hot[np.arange(len(y_train)), y_train] = 1

beat_feature_model.fit(np.array(X_train), y_one_hot, nb_epoch=10, batch_size=64) #, class_weight=class_weight)

Epoch 1/10
45168/45168 [==============================] - 19s - loss: 1.3045 - acc: 0.5300    
Epoch 2/10
45168/45168 [==============================] - 18s - loss: 0.9633 - acc: 0.6886    
Epoch 3/10
45168/45168 [==============================] - 18s - loss: 0.8733 - acc: 0.7013    
Epoch 4/10
45168/45168 [==============================] - 18s - loss: 0.8291 - acc: 0.7078    
Epoch 5/10
45168/45168 [==============================] - 19s - loss: 0.8019 - acc: 0.7132    
Epoch 6/10
45168/45168 [==============================] - 19s - loss: 0.7867 - acc: 0.7157    
Epoch 7/10
45168/45168 [==============================] - 19s - loss: 0.7732 - acc: 0.7202    
Epoch 8/10
45168/45168 [==============================] - 19s - loss: 0.7642 - acc: 0.7210    
Epoch 9/10
45168/45168 [==============================] - 19s - loss: 0.7573 - acc: 0.7233    
Epoch 10/10
45168/45168 [==============================] - 19s - loss: 0.7504 - acc: 0.7241    


In [74]:
y_one_hot = np.zeros((len(y_test_full), num_classes))
y_one_hot[np.arange(len(y_test_full)), y_test_full] = 1

beat_feature_model.evaluate(np.array(X_test_full), y_one_hot, batch_size=16)

61296/61325 [============================>.] - ETA: 0s

[0.51892312009578523, 0.82033428455244706]

In [63]:
y_one_hot = np.zeros((len(y_train_full), num_classes))
y_one_hot[np.arange(len(y_train_full)), y_train_full] = 1

beat_feature_model.fit(np.array(X_train_full), y_one_hot, nb_epoch=10, batch_size=64) #, class_weight=class_weight)

Epoch 1/10
540031/540031 [==============================] - 183s - loss: 0.3273 - acc: 0.8877   
Epoch 2/10
540031/540031 [==============================] - 182s - loss: 0.2719 - acc: 0.8963   
Epoch 3/10
540031/540031 [==============================] - 181s - loss: 0.2683 - acc: 0.8976   
Epoch 4/10
540031/540031 [==============================] - 172s - loss: 0.2658 - acc: 0.8986   
Epoch 5/10
540031/540031 [==============================] - 172s - loss: 0.2643 - acc: 0.8990   
Epoch 6/10
540031/540031 [==============================] - 173s - loss: 0.2626 - acc: 0.9000   
Epoch 7/10
540031/540031 [==============================] - 180s - loss: 0.2619 - acc: 0.9001   
Epoch 8/10
540031/540031 [==============================] - 181s - loss: 0.2609 - acc: 0.9002   
Epoch 9/10
540031/540031 [==============================] - 245s - loss: 0.2601 - acc: 0.9004   
Epoch 10/10
540031/540031 [==============================] - 201s - loss: 0.2592 - acc: 0.9008   


In [64]:
y_one_hot = np.zeros((len(y_test_full), num_classes))
y_one_hot[np.arange(len(y_test_full)), y_test_full] = 1

beat_feature_model.evaluate(np.array(X_test_full), y_one_hot, batch_size=16)

61280/61325 [============================>.] - ETA: 0s

[0.24706595587574895, 0.9108357113777158]

In [76]:
for song in ['Anubis', 'Bend Your Mind', 'Boogie Down', 'Bouff', 'Bubble Dancer']:
    step_song('In The Groove~{0}'.format(song), beat_feature_model, prediction_to_output_one_hot)